# Weather Information Agent

This project implements an intelligent weather assistant that can understand natural language queries about weather conditions in different locations. The agent leverages OpenAI's language models and the Weatherstack API to provide conversational responses about current weather conditions.

## Key Components
1. WeatherAgent : The core class that handles:
   
   - Location extraction from natural language queries
   - Weather data retrieval from Weatherstack API
   - Natural language response generation
   - Conversation context management for follow-up questions
2. MCP Integration : Implements the Machine Conversation Protocol to:
   
   - Expose weather functionality as a tool for the agent
   - Handle communication between the agent and external services
   - Process tool calls and responses
3. OpenAI Function Calling : Utilizes OpenAI's function calling capability to:
   
   - Allow the model to decide when to call the weather tool
   - Structure parameters for weather queries
   - Process tool results into conversational responses

## Features
- Natural Language Understanding : Processes queries like "What's the weather like in New York?" or "Will I need an umbrella tomorrow?"
- Context Awareness : Remembers previous locations for follow-up questions like "How about in London?"
- Comparative Queries : Handles questions about temperature comparisons and future conditions
- Conversational Responses : Generates friendly, informative responses rather than just raw data


### 1. Install necessary packages

In [2]:
!pip install openai python-dotenv nest_asyncio

### 2. Import Dependencies

In [ ]:
import os
import json
import requests
import asyncio
from dotenv import load_dotenv
from agents import Agent
from agents.mcp import MCPServer  # MCPServer
import nest_asyncio # to allow nested event loops (necessary for Jupyter notebooks)


nest_asyncio.apply()

### 3. Load the environment variables

In [14]:
load_dotenv()

# set up API keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')

### 4. Weather Agent Class
The WeatherAgent class is a Python implementation that provides weather information for a given location using natural language processing. It combines the OpenAI API for language understanding and the Weatherstack API for retrieving weather data.

### Key Components
#### Initialization
- The class requires API keys for both Weatherstack and OpenAI services
- These keys are used to authenticate API requests
#### Methods 

**1. call_openai_api**
- Makes requests to OpenAI's API
- Sends messages in a specific format required by OpenAI
- Returns the generated text response or None if an error occurs
- Handles API errors and exceptions gracefully 

**2. extract_location**
- Uses OpenAI to extract location information from a natural language query
- Provides specific instructions to the AI to focus only on location extraction
- Returns the extracted location name 

**3. get_weather_data**
- Fetches current weather data from Weatherstack API for a specified location
- Processes the API response to extract relevant weather information
- Returns a structured dictionary with key weather metrics including:
  - Location name and country
  - Temperature and "feels like" temperature
  - Humidity and wind speed
  - Weather description and cloud cover

**4. generate_weather_response**
- Creates a natural language description of weather conditions
- Uses OpenAI to convert structured weather data into conversational text
- Returns a friendly, informative weather description 

**5. process_weather_query**
- Orchestrates the entire weather information process
- Extracts location from user query
- Retrieves weather data for that location
- Generates a natural language response
- Handles errors at each step with appropriate fallback messages

In [15]:
# Step 2: Weather Agent Core Implementation
class WeatherAgent:
    def __init__(self, weather_api_key, openai_api_key):
        """
        Initialize the Weather Agent with API keys.
        
        Args:
            weather_api_key (str): API key for weather service
            openai_api_key (str): API key for OpenAI
        """
        self.weather_api_key = weather_api_key
        self.openai_api_key = openai_api_key
        self.conversation_context = {
            "last_location": None,
            "last_weather_data": None,
            "query_history": []
        }
    
    def call_openai_api(self, messages, model="gpt-3.5-turbo"):
        """
        Call OpenAI API directly using requests.
        
        Args:
            messages (list): List of message objects
            model (str): Model to use
            
        Returns:
            str: Response content
        """
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.openai_api_key}"
        }
        
        # Create the payload
        payload = {
            "model": model,
            "messages": messages,
            "temperature": 0.7
        }
        
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"].strip()
            else:
                print(f"Error calling OpenAI API: {response.status_code}")
                print(response.text)
                return None
        except Exception as e:
            print(f"Exception when calling OpenAI API: {e}")
            return None
            
    def extract_location(self, user_query):
        """
        Extract location from user query using OpenAI.
        
        Args:
            user_query (str): Natural language weather query
        
        Returns:
            str: Extracted location name or None if no location is found
        """
        # Add context from previous queries
        context_prompt = ""
        if self.conversation_context["last_location"]:
            context_prompt = f"Previous location mentioned was {self.conversation_context['last_location']}. "
        
        # Create system message with improved instructions
        system_content = (
            "You are a location extraction assistant. Extract the specific city or location from the given query. "
            "If the query doesn't mention a specific location but refers to a previous location, use that previous location. "
            "If the query is asking a comparative question like 'Is it warmer than yesterday?' or a future question like "
            "'Will I need an umbrella tomorrow?' without specifying a location, respond with 'USE_PREVIOUS_LOCATION'. "
            "If no location is mentioned and there's no previous context, respond with 'NO_LOCATION'."
        )
        
        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": f"{context_prompt}Extract the location from this query: '{user_query}'."}
        ]
        
        try:
            location = self.call_openai_api(messages)
            
            # Handle special responses
            if location == "USE_PREVIOUS_LOCATION":
                return self.conversation_context["last_location"]
            elif location == "NO_LOCATION":
                return None
            
            # Store the location in context
            self.conversation_context["last_location"] = location
            return location
        except Exception as e:
            print(f"Error extracting location: {e}")
            return None

### 5. Weather Agent Response Generation and Query Processing

In [16]:
# Step 3: Weather Data Retrieval and Response Generation
class WeatherAgent(WeatherAgent):
    def get_weather_data(self, location):
        """
        Fetch weather data for a given location using Weatherstack API.
        
        Args:
            location (str): City name or location identifier
        
        Returns:
            dict: Processed weather information
        """
        base_url = "http://api.weatherstack.com/current"
        params = {
            'access_key': self.weather_api_key,
            'query': location,
            'units': 'm'  # Metric units
        }
        
        try:
            response = requests.get(base_url, params=params)
            data = response.json()
            
            # Check for errors in the response
            if 'error' in data:
                print(f"Weather API error: {data['error']['info']}")
                return None
            
            # Extract relevant weather information
            weather_info = {
                'location': data['location']['name'],
                'country': data['location']['country'],
                'temperature': data['current']['temperature'],
                'feels_like': data['current']['feelslike'],
                'humidity': data['current']['humidity'],
                'description': data['current']['weather_descriptions'][0] if data['current']['weather_descriptions'] else 'No description available',
                'wind_speed': data['current']['wind_speed'],
                'cloudiness': data['current']['cloudcover']
            }
            
            # Store the weather data in context
            self.conversation_context["last_weather_data"] = weather_info
            return weather_info
        
        except Exception as e:
            print(f"Error fetching weather data: {e}")
            return None
    
    def generate_weather_response(self, weather_data, user_query):
        """
        Generate a natural language response for weather data.
        
        Args:
            weather_data (dict): Weather information
            user_query (str): Original user query
        
        Returns:
            str: Descriptive weather response
        """
        # Add context for comparative or future queries
        context_prompt = ""
        if "yesterday" in user_query.lower() or "warmer" in user_query.lower() or "colder" in user_query.lower():
            context_prompt = "This is a comparative question about temperature. "
        elif "tomorrow" in user_query.lower() or "umbrella" in user_query.lower() or "rain" in user_query.lower():
            context_prompt = "This is a question about future weather or precipitation. "
        
        messages = [
            {"role": "system", "content": f"You are a friendly weather narrator. Create a conversational weather description. {context_prompt}"},
            {"role": "user", "content": f"Create a friendly, informative weather description for this query: '{user_query}' using this data: {weather_data}"}
        ]
        
        try:
            response = self.call_openai_api(messages)
            return response
        except Exception as e:
            print(f"Error generating weather response: {e}")
            return None
    
    def process_weather_query(self, user_query):
        """
        Process a complete weather query from extraction to response.
        
        Args:
            user_query (str): Natural language weather query
        
        Returns:
            str: Comprehensive weather information response
        """
        # Store query in history
        self.conversation_context["query_history"].append(user_query)
        
        # Extract location
        location = self.extract_location(user_query)
        if not location:
            if self.conversation_context["last_location"]:
                # Use the last location if available
                location = self.conversation_context["last_location"]
                print(f"Using previous location: {location}")
            else:
                return "I need to know which location you're asking about. Could you specify a city or place?"
        
        # Get weather data
        weather_data = self.get_weather_data(location)
        if not weather_data:
            return f"Sorry, I couldn't retrieve weather data for {location}."
        
        # Generate natural language response
        weather_response = self.generate_weather_response(weather_data, user_query)
        return weather_response or "I encountered an issue generating the weather description."

### 6. MCP Server Implementation

The WeatherMCPServer class that extends MCPServer to:
- Create and manage a WeatherAgent instance
- Implement required methods like connect , cleanup , call_tool , and list_tools
- Expose the weather functionality as a tool that can be called by the agent

In [17]:
# MCP Server Implementation for Weather
class WeatherMCPServer(MCPServer):
    def __init__(self, weather_agent=None):
        super().__init__()
        self.weather_agent = weather_agent or WeatherAgent(
            weather_api_key=WEATHER_API_KEY, 
            openai_api_key=OPENAI_API_KEY
        )
    
    @property
    def name(self):
        """Return the name of the server."""
        return "weather_server"
    
    async def connect(self):
        """Connect to the server."""
        # No actual connection needed for this example
        return True
    
    async def cleanup(self):
        """Clean up resources."""
        # No cleanup needed for this example
        pass
    
    async def call_tool(self, tool_name, tool_params):
        """Call a tool by name with parameters."""
        if tool_name == "weather_tool":
            return self.weather_agent.process_weather_query(tool_params["query"])
        raise ValueError(f"Unknown tool: {tool_name}")
    
    async def list_tools(self):
        """List available tools."""
        # Create a weather tool using our custom FunctionTool
        weather_tool = FunctionTool(
            name="weather_tool",
            description="Get current weather information for a location",
            function=lambda params: self.weather_agent.process_weather_query(params["query"]),
            parameters={
                "query": {
                    "type": "string",
                    "description": "The location or weather query to process"
                }
            },
            required=["query"]
        )
        return [weather_tool]

# Create a persistent instance of the MCP server
weather_mcp_server = WeatherMCPServer()

### 7. Function Tool Implementation

A custom FunctionTool class that represents a tool that can be called by the agent


In [ ]:
class FunctionTool:
    """A simple implementation of FunctionTool"""
    def __init__(self, name, description, function, parameters, required=None):
        self.name = name
        self.description = description
        self.function = function
        self.parameters = parameters
        self.required = required or []
    
    def __str__(self):
        return f"FunctionTool(name={self.name})"
    
    def to_dict(self):
        """Convert the tool to a dictionary for easy use"""
        return {
            "name": self.name,
            "description": self.description,
            "parameters": self.parameters,
            "required": self.required
        }

# Proper MCP Server implementation for weather
class WeatherMCPServer(MCPServer):
    def __init__(self, weather_agent=None):
        super().__init__()
        self.weather_agent = weather_agent or WeatherAgent(
            weather_api_key=WEATHER_API_KEY, 
            openai_api_key=OPENAI_API_KEY
        )
    
    @property
    def name(self):
        """Return the name of the server."""
        return "weather_server"
    
    async def connect(self):
        """Connect to the server."""
        # No actual connection needed for this example
        return True
    
    async def cleanup(self):
        """Clean up resources."""
        # No cleanup needed for this example
        pass
    
    async def call_tool(self, tool_name, tool_params):
        """Call a tool by name with parameters."""
        if tool_name == "weather_tool":
            return self.weather_agent.process_weather_query(tool_params["query"])
        raise ValueError(f"Unknown tool: {tool_name}")
    
    async def list_tools(self):
        """List available tools."""
        # Create a weather tool using our custom FunctionTool
        weather_tool = FunctionTool(
            name="weather_tool",
            description="Get current weather information for a location",
            function=lambda params: self.weather_agent.process_weather_query(params["query"]),
            parameters={
                "query": {
                    "type": "string",
                    "description": "The location or weather query to process"
                }
            },
            required=["query"]
        )
        return [weather_tool]

# Create a persistent instance of the MCP server
weather_mcp_server = WeatherMCPServer()

### 7. OpenAI Tools Client and Agent-MCP Integration

This cell implements:

1. OpenAIToolsClient : A client for the OpenAI API that:
   
   - Converts MCP tools to the OpenAI function calling format
   - Handles calling the OpenAI API with tools
   - Processes tool calls and results
   
2. AgentMCPIntegration : A class that integrates the agent with the MCP server:
   
   - Creates an agent instance
   - Processes messages using the OpenAI client and MCP tools
   - Handles the flow of information between the user, agent, and tools
This integration allows the agent to use the weather functionality through the OpenAI function calling interface.

In [ ]:
# ppenAI tools client and agent-MCP Integration
class OpenAIToolsClient:
    def __init__(self, api_key):
        self.api_key = api_key
    
    async def get_tools_from_mcp_server(self, server):
        """Get tools from an MCP server in OpenAI format"""
        tools = await server.list_tools()
        openai_tools = []
        
        for tool in tools:
            openai_tool = {
                "type": "function",
                "function": {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": {
                        "type": "object",
                        "properties": tool.parameters,
                        "required": tool.required
                    }
                }
            }
            openai_tools.append(openai_tool)
        
        return openai_tools
    
    async def call_with_tools(self, messages, tools=None):
        """Call OpenAI API with tools"""
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        
        payload = {
            "model": "gpt-3.5-turbo",
            "messages": messages,
            "temperature": 0.7
        }
        
        if tools:
            payload["tools"] = tools
        
        try:
            response = requests.post(
                "https://api.openai.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error calling OpenAI API: {response.status_code}")
                print(response.text)
                return None
        except Exception as e:
            print(f"Exception when calling OpenAI API: {e}")
            return None
    
    async def process_with_mcp_server(self, messages, server):
        """Process messages using tools from an MCP server"""
        # Get tools from the MCP server in OpenAI format
        tools = await self.get_tools_from_mcp_server(server)
        
        # Call OpenAI with the tools
        response = await self.call_with_tools(messages, tools)
        
        # If we got a response and it contains a tool call
        if response and "choices" in response and response["choices"]:
            message = response["choices"][0]["message"]
            
            # Check if the message contains tool calls
            if "tool_calls" in message and message["tool_calls"]:
                tool_calls = message["tool_calls"]
                
                # Process each tool call
                for tool_call in tool_calls:
                    function_call = tool_call["function"]
                    tool_name = function_call["name"]
                    tool_params = json.loads(function_call["arguments"])
                    
                    # Call the tool through the MCP server
                    tool_result = await server.call_tool(tool_name, tool_params)
                    
                    # Add the tool call and result to the messages
                    messages.append({
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [tool_call]
                    })
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call["id"],
                        "content": str(tool_result)
                    })
                
                # Get the final response that incorporates the tool results
                final_response = await self.call_with_tools(messages)
                
                if final_response and "choices" in final_response:
                    return final_response["choices"][0]["message"]["content"]
            
            # If no tool calls, return the content directly
            elif "content" in message:
                return message["content"]
        
        return "I'm sorry, I couldn't process your request."

# Integration class that simulates the Agent-MCP integration
class AgentMCPIntegration:
    def __init__(self, name, mcp_server, openai_api_key):
        """
        Initialize the Agent-MCP integration
        
        Args:
            name (str): Name of the agent
            mcp_server (MCPServer): MCP server to use
            openai_api_key (str): OpenAI API key
        """
        self.name = name
        self.mcp_server = mcp_server
        self.openai_api_key = openai_api_key
        self.openai_client = OpenAIToolsClient(openai_api_key)
        
        # Create an Agent instance just for show (we won't use its methods)
        try:
            self.agent = Agent(name=name)
            print(f"Created Agent with name '{name}'")
        except Exception as e:
            print(f"Could not create Agent: {e}")
            self.agent = None
    
    async def process_message(self, message):
        """
        Process a message using the MCP server's tools
        
        Args:
            message (str): User message
            
        Returns:
            str: Response
        """
        # Create a messages array with system instructions and user message
        messages = [
            {
                "role": "system", 
                "content": "You are a helpful assistant that can answer questions about the weather. Use the weather_tool to get weather information."
            },
            {
                "role": "user",
                "content": message
            }
        ]
        
        # Process the message using our OpenAI client with MCP tools
        response = await self.openai_client.process_with_mcp_server(messages, self.mcp_server)
        
        return response

# Create an AgentMCPIntegration instance
agent_mcp = AgentMCPIntegration(
    name="WeatherAssistant",
    mcp_server=weather_mcp_server,
    openai_api_key=OPENAI_API_KEY
)

Created Agent with name 'WeatherAssistant'


In [ ]:

# Process a weather query with the Agent-MCP integration
async def process_weather_query(query):
    """Process a weather query using Agent-MCP integration"""
    return await agent_mcp.process_message(query)

# Helper function to run a single query
async def run_query(query):
    """Helper function to run a single query"""
    print(f"\nQuery: {query}")
    try:
        result = await process_weather_query(query)
        print(f"Response: {result}")
        return result
    except Exception as e:
        print(f"Error processing query: {e}")
        return f"Error: {str(e)}"

# Process a series of queries to demonstrate functionality
async def run_demo():
    """Process a series of demo queries"""
    queries = [
        "What's the weather like in New York today?",
        "How about in London?",
        "Is it warmer than yesterday?",
        "Will I need an umbrella tomorrow?"
    ]
    
    for query in queries:
        await run_query(query)

# For Jupyter notebook usage
def start_demo():
    """Start the demo in a way that works in Jupyter notebooks"""
    print("Starting weather agent demo...")
    
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(run_demo())
    except Exception as e:
        print(f"Demo failed: {e}")
        print("\nTIP: Make sure to run all cells in order before running this demo.")

# When run directly
if __name__ == "__main__":
    start_demo()

Starting weather agent demo...

Query: What's the weather like in New York today?
Response: The weather in New York today is 13°C with partly cloudy skies. It might feel slightly cooler at 12°C due to the wind blowing at 18 km/h. Enjoy the mix of sun and clouds!

Query: How about in London?
Response: In London, the current temperature is 12°C with partly cloudy skies and a moderate wind speed of 15 km/h. It's a good day to enjoy some outdoor activities or relax indoors.

Query: Is it warmer than yesterday?
Response: Yes, it looks like today is slightly warmer than yesterday in London. Today's temperature is 12 degrees Celsius, while yesterday's temperature was also around 12 degrees Celsius. However, it feels just a touch cooler today at 11 degrees with a bit of a breeze.

Query: Will I need an umbrella tomorrow?
Response: It looks like the weather tomorrow in London, United Kingdom will have partly cloudy skies with a temperature of 12°C. There is no mention of rain, so you should not